# Initialization

In [3]:
%reload_ext autoreload
%autoreload 2

import os, sys
from importlib import reload

import numpy as np
import scipy as sp
import scipy.stats
import corner
from nose.tools import assert_true, assert_false, assert_raises

import matplotlib as mpl
import matplotlib.pyplot as plt

import kdes

# Kernel Base Class

In [4]:
aa = np.arange(16).reshape(4, 4)
bb = [1, 2]
print(aa[bb, bb])

[ 5 10]
